# 지난 시간 복습
1. 복잡한 계산 그래프(이론)
2. 복잡한 계산 그래프(구현)
3. 메모리 관리와 순환 참조
4. 메모리 절약 모드

1,2 는 **_유향그래프의 위상 정렬_** 에 관한 내용  
3,4 는 파이썬 언어의 **_GC_** 에 관해서, 그리고 **_메모리 개선_** 을 구현  
특히 4에서는 with no_grad()와 비슷한 효과를 낼 수 있도록, with구문을 이용하는 방법을 설명  

In [37]:
import numpy as np
import weakref
import contextlib
from typing import List, Tuple


class Config:
    enable_backprop = True

def as_array(data):
    if np.isscalar(data):
        return np.array(data)
    return data

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config("enable_backprop", False)

class Variable:
    def __init__(self, data:np.array):
        if not data:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def clean_grad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            xs, ys = f.inputs, f.outputs
            gys = [y().grad for y in ys]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,

            for x, gx in zip(xs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x. grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in ys:
                    y().grad = None

class Function:
    def __call__(self, *inputs:List[Variable]):
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs])
        for output in outputs:
            output.set_creator(self)
        self.outputs = [weakref.ref(output) for output in outputs]
        self.inputs = inputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        return NotImplementedError()
    
    def backward(self, gy):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        return gy * np.exp(self.inputs[0].data)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        return gy * 2 * self.inputs[0].data
    
class Add(Function):
    def forward(self, x0, x1):
        return x0 + x1,

    def backward(self, gy):
        return gy, gy

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [38]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

y = square(x0)
print(y.data)

y = exp(x1)
print(y.data)

y = add(square(x0), x1)
print(y.data)

5
4
20.085536923187668
7


In [39]:
x = Variable(np.array(2.00))
a = square(x)
y = add(square(a), square(a))
y.backward()
print(y.data)
print(x.grad)

32.0
64.0


In [40]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [41]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

# 이번주에는 총 2가지를 배운다 (step19~22)
1. 변수 사용성 개선
2. 메서드 오버로딩 1 & 2

# Not Clone coding

In [66]:
import numpy as np
import weakref
import contextlib
from typing import List, Tuple


class Config:
    enable_backprop = True

def as_array(data):
    if np.isscalar(data):
        return np.array(data)
    return data

def as_varialbe(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config("enable_backprop", False)

class Variable:
    def __init__(self, data:np.array, name=None):
        if not data:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        self.name = name

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def clean_grad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            xs, ys = f.inputs, f.outputs
            gys = [y().grad for y in ys]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,

            for x, gx in zip(xs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x. grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in ys:
                    y().grad = None

    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def shape(self):
        return self.data.shape
    
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ")"
    
    def __mul__(self, other):
        return mul(self, other)
    
    def __add__(self, other):
        return add(self, other)
    
    def __rmul__(self, other):
        return mul(other, self)
    
    def __radd__(self, other):
        return add(other, self)
    
    def __neg__(self):
        return neg(self)
    
    def __sub__(self, other):
        return sub(self, other)
    
    def __rsub__(self, other):
        return sub(other, self)
    
    def __truediv__(self, other):
        return div(self, other)
    
    def __rtruediv__(self, other):
        return div(other, self)
    
    def __pow__(self, c):
        return pow(self, c)


class Function:
    def __call__(self, *inputs:List[Variable]):
        inputs = [as_varialbe(as_array(input)) for input in inputs]

        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs])
        for output in outputs:
            output.set_creator(self)
        self.outputs = [weakref.ref(output) for output in outputs]
        self.inputs = inputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        return NotImplementedError()
    
    def backward(self, gy):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        return gy * np.exp(self.inputs[0].data)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        return gy * 2 * self.inputs[0].data
    
class Add(Function):
    def forward(self, x0, x1):
        #x1 = as_array(x1)
        return x0 + x1,

    def backward(self, gy):
        return gy, gy
    
class Mul(Function):
    def forward(self, x0, x1):
        #x1 = as_array(x1)
        return x0 * x1
    
    def backward(self, gy):
        return gy * self.inputs[1].data, gy * self.inputs[0].data
    
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
class Sub(Function):
    def forward(self, x0, x1):
        return x0 - x1
    
    def backward(self, gy):
        return gy, -gy
    
class Div(Function):
    def forward(self, x0, x1):
        return x0 / x1
    
    def backward(self, gy):
        return gy / x1, -gy * self.inputs[0].data / (self.inputs[1].data ** 2)
    
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        return x ** self.c
    
    def backward(self, gy):
        return gy * self.c * self.inputs[0].data ** (self.c - 1)

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

def mul(x0, x1):
    return Mul()(x0, x1)

def neg(x):
    return Neg()(x)

def sub(x0, x1):
    return Sub()(x0, x1)

def div(x0, x1):
    return Div()(x0, x1)

def pow(x, c):
    return Pow(c)(x)

In [63]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c

y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [64]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

try:
    x = Variable(np.array(2.0))
    y = 3.0 + x
    print(y)
except Exception as e:
    print(f"{e}")

variable(5.0)
variable(5.0)


In [67]:
x = Variable(np.array(2))
y = -x
print(y)

y1 = 2.0 - x
y2 = x - 2.0
print(y1)
print(y2)

y1 = 2.0 / x
y2 = x / 2.0
print(y1)
print(y2)

y1 = x ** 2
print(y1)

variable(-2)
variable(0.0)
variable(0.0)
variable(1.0)
variable(1.0)
variable(4)


# 1. 변수 사용성 개선

Variable의 이름, 사이즈, 길이, dtype 등을 @property로 속성처럼 부를 수 있게 구현  
또한, len, print도 사용할 수 있게끔 개선

In [5]:
import numpy as np
import weakref
import contextlib
from typing import List, Tuple


class Config:
    enable_backprop = True

def as_array(data):
    if np.isscalar(data):
        return np.array(data)
    return data

def as_varialbe(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config("enable_backprop", False)

class Variable:
    def __init__(self, data:np.array, name=None):
        if not data:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        self.name = name

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def clean_grad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            xs, ys = f.inputs, f.outputs
            gys = [y().grad for y in ys]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,

            for x, gx in zip(xs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x. grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in ys:
                    y().grad = None

    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def shape(self):
        return self.data.shape
    
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ")"

class Function:
    def __call__(self, *inputs:List[Variable]):
        inputs = [as_varialbe(as_array(input)) for input in inputs]

        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs])
        for output in outputs:
            output.set_creator(self)
        self.outputs = [weakref.ref(output) for output in outputs]
        self.inputs = inputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        return NotImplementedError()
    
    def backward(self, gy):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        return gy * np.exp(self.inputs[0].data)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        return gy * 2 * self.inputs[0].data
    
class Add(Function):
    def forward(self, x0, x1):
        #x1 = as_array(x1)
        return x0 + x1,

    def backward(self, gy):
        return gy, gy

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

In [9]:
x = Variable(np.array([2.0]))

print(x)
print(x.size)
print(x.dtype)
print(x.size)
print(x.shape)

variable([2.])
1
float64
1
(1,)


# 2. 메서드 오버로딩 1 & 2
  
Variable에 -, neg, +, *, ** 등을 사용할 수 있게끔 메서드 오버로딩

In [1]:
import numpy as np
import weakref
import contextlib
from typing import List, Tuple


class Config:
    enable_backprop = True

def as_array(data):
    if np.isscalar(data):
        return np.array(data)
    return data

def as_varialbe(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config("enable_backprop", False)

class Variable:
    def __init__(self, data:np.array, name=None):
        if not data:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        self.name = name

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def clean_grad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            xs, ys = f.inputs, f.outputs
            gys = [y().grad for y in ys]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = gxs,

            for x, gx in zip(xs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x. grad = x.grad + gx
            
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in ys:
                    y().grad = None

    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def shape(self):
        return self.data.shape
    
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ")"
    
    def __mul__(self, other):
        return mul(self, other)
    
    def __add__(self, other):
        return add(self, other)
    
    def __rmul__(self, other):
        return mul(other, self)
    
    def __radd__(self, other):
        return add(other, self)
    
    def __neg__(self):
        return neg(self)
    
    def __sub__(self, other):
        return sub(self, other)
    
    def __rsub__(self, other):
        return sub(other, self)
    
    def __truediv__(self, other):
        return div(self, other)
    
    def __rtruediv__(self, other):
        return div(other, self)
    
    def __pow__(self, c):
        return pow(self, c)


class Function:
    def __call__(self, *inputs:List[Variable]):
        inputs = [as_varialbe(as_array(input)) for input in inputs]

        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = ys,
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([input.generation for input in inputs])
        for output in outputs:
            output.set_creator(self)
        self.outputs = [weakref.ref(output) for output in outputs]
        self.inputs = inputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, x):
        return NotImplementedError()
    
    def backward(self, gy):
        return NotImplementedError()
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        return gy * np.exp(self.inputs[0].data)
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        return gy * 2 * self.inputs[0].data
    
class Add(Function):
    def forward(self, x0, x1):
        #x1 = as_array(x1)
        return x0 + x1,

    def backward(self, gy):
        return gy, gy
    
class Mul(Function):
    def forward(self, x0, x1):
        #x1 = as_array(x1)
        return x0 * x1
    
    def backward(self, gy):
        return gy * self.inputs[1].data, gy * self.inputs[0].data
    
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
class Sub(Function):
    def forward(self, x0, x1):
        return x0 - x1
    
    def backward(self, gy):
        return gy, -gy
    
class Div(Function):
    def forward(self, x0, x1):
        return x0 / x1
    
    def backward(self, gy):
        return gy / x1, -gy * self.inputs[0].data / (self.inputs[1].data ** 2)
    
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        return x ** self.c
    
    def backward(self, gy):
        return gy * self.c * self.inputs[0].data ** (self.c - 1)

def exp(x):
    return Exp()(x)

def square(x):
    return Square()(x)

def add(x0, x1):
    return Add()(x0, x1)

def mul(x0, x1):
    return Mul()(x0, x1)

def neg(x):
    return Neg()(x)

def sub(x0, x1):
    return Sub()(x0, x1)

def div(x0, x1):
    return Div()(x0, x1)

def pow(x, c):
    return Pow(c)(x)

In [2]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = a * b + c

y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [3]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

try:
    x = Variable(np.array(2.0))
    y = 3.0 + x
    print(y)
except Exception as e:
    print(f"{e}")

variable(5.0)
variable(5.0)


In [4]:
x = Variable(np.array(2))
y = -x
print(y)

y1 = 2.0 - x
y2 = x - 2.0
print(y1)
print(y2)

y1 = 2.0 / x
y2 = x / 2.0
print(y1)
print(y2)

y1 = x ** 2
print(y1)

variable(-2)
variable(0.0)
variable(0.0)
variable(1.0)
variable(1.0)
variable(4)
